In [ ]:
import keras
from keras.optimizers import Adam
from synDS import propertiesScanner
from synDS.generator import Generator
from Score.helpers import *
from Score.algorithms import trustworthiness
tf.compat.v1.disable_eager_execution()

In [ ]:
# Todo: Specify evaluation scenario & decide if you want MUST or MAY
scenario = "creditrisk" # diabetes or creditrisk
generate_synDS = False # set to "True" for first execution, False to speed up evaluation if synDS already exists
must = True # set to "False" to generate and compute Advanced Score MAY

In [ ]:
# ********** Load respective model and provide info about features ************
if scenario == "diabetes":
        if must:
                solution = "Solutions/S1DiabetesPredictionRF/AdvancedScoreMUST"
        else:
                solution = "Solutions/S1DiabetesPredictionRF/AdvancedScoreMAY"
        model = pickle.load(open(f"{solution}/model.pkl", "rb")) # RF
        target_name = "diabetes"
        ds_name = "diabetes_prediction_dataset"
        categorical_name = ["gender", "smoking_history"]
        data_types = ['categorical', 'discrete', 'categorical', 'categorical', 'categorical', 'continuous', 'continuous', 'discrete']
elif scenario == "creditrisk":
        if must:
                solution = "Solutions/S2HomeCreditDefaultRiskDNN/AdvancedScoreMUST"
        else:
                solution = "Solutions/S2HomeCreditDefaultRiskDNN/AdvancedScoreMAY"
        model = keras.models.load_model(f"{solution}/model.keras") # DNN
        optimizer = Adam(learning_rate=0.001) # DNN
        model.compile(loss="categorical_crossentropy", optimizer=optimizer) # DNN
        target_name = "TARGET"
        ds_name = "credit_risk"
        categorical_name = ["CODE_GENDER","FLAG_OWN_CAR","FLAG_OWN_REALTY","NAME_EDUCATION_TYPE"]
        data_types = ['categorical', 'categorical', 'categorical', 'continuous', 'continuous', 'categorical', 'discrete', 'discrete']
else:
        raise AssertionError("Check specifications: Evaluation scenario does not exist")

In [ ]:
# ********** Get MUST Statistical Properties From User **************
if generate_synDS:
        data = pd.read_csv(f"{'/'.join(solution.split('/')[0:-1])}/TrueScore/{ds_name}.csv")
        data = data.drop(target_name, axis=1)
        feature_names, min_, max_, unique_values, mean, std, skew = propertiesScanner.scan(data)

# ********** Generate And Label The Synthetic Dataset  ************
        generator = Generator()
        if must:
                synDS_must = generator.generate_dataset_must(feature_names, data_types, solution, min_, max_, unique_values)
                synDS = generator.label_synDS(model, synDS_must, categorical_name, target_name, True)
                synDS.to_csv(f"{solution}/synDS_must.csv", index=False)
        else:
                synDS_may = generator.generate_dataset_may(feature_names, data_types, solution, min_, max_, unique_values, mean=mean, std=std, skew=skew)
                synDS = generator.label_synDS(model, synDS_may, categorical_name, target_name, False)
                synDS.to_csv(f"{solution}/synDS_may.csv", index=False)
train = None
if must:
        test = pd.read_csv(f"{solution}/synDS_must.csv")
else:
        test = pd.read_csv(f"{solution}/synDS_may.csv")

In [ ]:
# ********** Compute Score ************
factsheet = json.load(open(f"{solution}/factsheet.json"))
config_fairness = json.load(open(f"Score/configs/mappings/fairness/default.json"))
config_exp = json.load(open(f"Score/configs/mappings/explainability/default.json"))
config_rob = json.load(open(f"Score/configs/mappings/robustness/default.json"))
config_met = json.load(open(f"Score/configs/mappings/methodology/default.json"))
config_weights = json.load(open(f"Score/configs/weights/default.json"))
solution_set_path = f"{solution}/"
mappings_config = {}
mappings_config["fairness"] = config_fairness
mappings_config["explainability"] = config_exp
mappings_config["robustness"] = config_rob
mappings_config["methodology"] = config_met
result = trustworthiness.get_final_score(model, train, test, config_weights, mappings_config, factsheet, solution_set_path, recalc=False)
must_final_scores, must_results, must_properties = result

In [ ]:
# ********** Plotting for Report ************
from Score.plot import *

trust_score = trustworthiness.get_trust_score(must_final_scores, config_weights["pillars"])

data = {"final_score":must_final_scores,
        "results":must_results,
        "trust_score":trust_score,
        "properties" : must_properties}

final_score = data["final_score"]
pillars = list(final_score.keys())
values = list(final_score.values())
pillar_colors = ['#06d6a0', '#ffd166', '#ef476f', '#118ab2']
title = "Overall Trust Score {}/5".format(trust_score)
figname = "overall.png"

draw_bar_plot(pillars, values, pillar_colors, title, figname, solution)
draw_pillar_scores(data['results'], final_score, pillar_colors, solution)